<a href="https://colab.research.google.com/github/prakhar9998/image-denoising/blob/main/denoise_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install flask-ngrok

In [72]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [73]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2 as cv

In [74]:
def psnr(im1, im2):
    return tf.image.psnr(im1, im2, max_val=1.0)

model = tf.keras.models.load_model('gdrive/MyDrive/denoise/dncnn_model', compile=False, custom_objects={'psnr': psnr})

In [75]:
def dcnn_loss(predicted, true):
    return tf.keras.backend.sum(tf.keras.backend.square(predicted - true)) / 2

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=dcnn_loss, metrics=[psnr])

In [76]:
def predict(img):
    # expects a PIL image object
    plt.imshow(img)
    plt.show()

    img = tf.keras.preprocessing.image.img_to_array(img) / 255
    print(img.shape)
    patches = tf.image.extract_patches(images=np.expand_dims(img, 0),
                         sizes=[1,140,140,1],
                         strides=[1,140,140,1],
                         rates=[1,1,1,1],
                         padding='SAME')
    
    patches = tf.reshape(patches, [-1, 140, 140, 3])

    res_images = []
    print("Total patches: ", len(patches))
    for i in range(len(patches)):
        print("Predicting patch no. ", i + 1)
        res_images.append(model.predict(np.expand_dims(patches[i], 0)))

    res_patches = np.array(res_images)
    res_patches = tf.reshape(res_patches, [-1, 140, 140, 3])

    test = tf.split(res_patches, res_patches.shape[0])

    horizontal_patches = math.ceil(img.shape[0] / 140)
    vertical_patches = math.ceil(img.shape[1] / 140)

    combined = []

    for i in range(horizontal_patches):
        combined.append(tf.concat([test[i * vertical_patches + j][0] for j in range(vertical_patches)], 1))

    final = tf.concat(combined, 0)

    output_image = tf.keras.preprocessing.image.array_to_img(final)

    return output_image

In [77]:
def denoise_opencv(pil_image):
    img = np.array(pil_image) 
    # Convert RGB to BGR 
    img = img[:, :, ::-1].copy() 
    dst = cv.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    color_coverted = cv.cvtColor(dst, cv.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_coverted)
    return pil_image

In [ ]:
img1 = tf.keras.preprocessing.image.load_img('gdrive/MyDrive/denoise/noisy1.png')
imgres = denoise_opencv(img1)
plt.imshow(imgres)
plt.show()

In [ ]:
from flask import Flask
from flask import request
from flask import send_file
from flask_ngrok import run_with_ngrok
import io
from PIL import Image

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# for / root, return Hello Word
@app.route("/")
def root():
    url = request.method
    return f"Hello World! {url}"

@app.route("/denoise", methods=['POST'])
def denoise():
    img_requested = request.files['image'].read()
    denoise_method = "CNN"
    try:
        denoise_method = request.form['method']
        if denoise_method != "NLM":
            denoise_method = "CNN"
    except KeyError:
        denoise_method = "CNN"

    img = Image.open(io.BytesIO(img_requested))
    img = img.convert("RGB")

    if denoise_method is "CNN":
        denoised_img = predict(img)
    else:
        denoised_img = denoise_opencv(img)

    file_object = io.BytesIO()

    denoised_img.save(file_object, "png")

    file_object.seek(0)

    return send_file(file_object, mimetype='image/png')

app.run()